In [16]:
import os
import pandas as pd
import numpy as np
from utils import to_21d_vec, get_beat_vector
import re
import sys
sys.path.insert(1, './harmalysis')
from inputRN2Chord import inputRN

In [17]:
script_dir = os.getcwd()
score_rel_path = "../datasets/haydn_op20_harm/haydn_reduced_score_for_vec/6_iv_score_upm3_trans_vector_pre.krn"
scorepath = os.path.join(script_dir, score_rel_path)


In [18]:
df = pd.read_csv(scorepath, sep="\t", header=None)
pd.set_option('display.max_rows', df.shape[0]+1)
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]
df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]

#sear org
#df.columns = ["voice4", "voice3", "voice2", "voice1", "harm", "beat", "measure", "key", "meter" ]

In [19]:
#process dataframe, delete bars, score representations, and "rest slice"
df = df[~df['beat'].astype(str).str.startswith(('=','.','*'))]
df

,harm,voice4,voice3,voice2,voice1,index,beat,measure,key,meter
2,I,1r,1r,2.r,8gg#,25,1.5,.,C#,M4/4
3,I,1r,1r,2.r,8gg#,26,2,.,C#,M4/4
4,I,1r,1r,2.r,8gg#,27,2.5,.,C#,M4/4
5,I,1r,1r,2.r,8ee#,28,3,.,C#,M4/4
7,I,1r,1r,2.r,16ee#,30,3.75,.,C#,M4/4
8,(I),1r,1r,[4cc#,8cc#,31,4,.,C#,M4/4
9,(I),1r,1r,[4cc#,8ee#,32,4.5,.,C#,M4/4
11,(I),1r,1r,4cc#],8dd#,34,1,=2,C#,M4/4
12,(I),1r,1r,4cc#],4ddd#,35,1.5,=2,C#,M4/4
13,Vb,1r,1r,2b#,4ddd#,36,2,=2,C#,M4/4


In [20]:

#transform to vectors first
prev_note_list = []
all_vecs = []
for index, row in df.iterrows():
    
    # for this onset slice 
    this_onset_vec = []

    ######################################### Process notes and onsets ########################################
    voices = ["voice4","voice3","voice2","voice1"]
    cur_note_tmp = []
    for v in voices:
        this_note = ''.join(row[[v]].values)
        if ' ' in this_note: #for a part has multiple voices
            this_note_list = this_note.split(" ")
            cur_note_tmp.extend(this_note_list)
        else:
            cur_note_tmp.append(this_note)

    cur_note_list = []
    for n in cur_note_tmp:
        note_name = re.sub('[^a-gA-G#-]+', '', n)
        if note_name:
            cur_note_list.append(note_name)

    cur_onset_list = list(set(cur_note_list) - set(prev_note_list))
    prev_note_list = cur_note_list


    note_vec = to_21d_vec(cur_note_list)
    onset_vec = to_21d_vec(cur_onset_list)
    this_onset_vec.extend(note_vec)
    this_onset_vec.extend(onset_vec)

    ######################################### Process beat position ########################################
    beat_pos = ''.join(row[["beat"]].values)
    meter = ''.join(row[["meter"]].values)
    meter = meter.replace("M", "")
    beat_vec = get_beat_vector(beat_pos, meter)
    this_onset_vec.extend(beat_vec)
    
    ######################################### Process chord label ########################################
    harm = ''.join(row[["harm"]].values)
    harm = re.sub('[()]', '', harm) # process string
    key = ''.join(row[["key"]].values)
    key_harm = key + ":" + harm
    #print(key_harm)
    try:
        chord_label = inputRN(key_harm)["Chord label"]
    except:
        continue # pass this onset slice if RN is not recognizable
    this_onset_vec.append(str(chord_label))
    this_onset_vec.append(''.join(row[['measure']].values))
    all_vecs.append(this_onset_vec)




In [21]:
def get_mea_num(measure):
    if measure == '.':
        measure = 1
    else:
        measure = int(re.findall("\d+", measure)[0])
    return measure

def get_window_vecs(all_vecs, head_idx, this_mea, window_size):
    window_vec = []
    window_onsets_list = []
    window_chords_list = []
    
    next_tail_mea = this_mea
    tail_idx = head_idx
    while next_tail_mea <= this_mea + window_size - 1:
        this_tail_mea = get_mea_num(all_vecs[tail_idx][-1])
        this_onset = all_vecs[tail_idx][:-2]
        window_onsets_list.append(this_onset)
        
        this_chord = all_vecs[tail_idx][-2]
        if not window_chords_list:
            window_chords_list.append(this_chord)
        elif window_chords_list and this_chord != window_chords_list[-1]:
            window_chords_list.append(this_chord)
        
        if tail_idx+1 < len(all_vecs):
            tail_idx += 1
        #print(tail_idx)
        next_tail_mea = get_mea_num(all_vecs[tail_idx][-1])
        #print(next_tail_mea)
    
    window_vec.append(window_onsets_list)
    window_vec.append(window_chords_list)
    return window_vec


In [22]:
######################################## Process window #####################################
window_size = 2
#hop_size = 1
head_mea = 0
max_mea = get_mea_num(''.join(df['measure'].iloc[-1]))

pre_mea = 1

### cur_window_list = [[window_onsets_list][window_chords_list]]
all_windows_vec = []

for idx, vec in enumerate(all_vecs):
    print("index", idx)
    this_mea = get_mea_num(vec[-1])
    
    if this_mea != pre_mea:
        #new measure -> proceed to a new window
        this_window_vec = get_window_vecs(all_vecs, idx, this_mea, window_size)
        all_windows_vec.append(this_window_vec)
        print(this_mea, this_window_vec)
        
    if this_mea == max_mea - window_size + 1:
        break
    pre_mea = this_mea
#print(all_windows_vec)

index 0
index 1
index 2
index 3
index 4
index 5
index 6
index 7
2 [[[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

KeyboardInterrupt: 

In [69]:
mea_window_size = 2
hop_size = 1
head_mea = 0
max_mea = 
window_number = 1
pre_mea = 1

### cur_window_list = [[window_onsets_list][window_chords_list]]
all_windows = []
cur_window_list = []
window_chords_list = []
window_onsets_list = []

prev_note_list = []
for index, row in df.iterrows():
    
    # for this onset slice 
    this_onset_vec = []

    ######################################### Process notes and onsets ########################################
    voices = ["voice4","voice3","voice2","voice1"]
    cur_note_tmp = []
    for v in voices:
        this_note = ''.join(row[[v]].values)
        if ' ' in this_note: #for a part has multiple voices
            this_note_list = this_note.split(" ")
            cur_note_tmp.extend(this_note_list)
        else:
            cur_note_tmp.append(this_note)

    cur_note_list = []
    for n in cur_note_tmp:
        note_name = re.sub('[^a-gA-G#-]+', '', n)
        if note_name:
            cur_note_list.append(note_name)

    cur_onset_list = list(set(cur_note_list) - set(prev_note_list))
    prev_note_list = cur_note_list


    note_vec = to_21d_vec(cur_note_list)
    onset_vec = to_21d_vec(cur_onset_list)
    this_onset_vec.extend(note_vec)
    this_onset_vec.extend(onset_vec)

    ######################################### Process beat position ########################################
    beat_pos = ''.join(row[["beat"]].values)
    meter = ''.join(row[["meter"]].values)
    meter = meter.replace("M", "")
    beat_vec = get_beat_vector(beat_pos, meter)
    this_onset_vec.extend(beat_vec)
    
    ######################################### Process chord label ########################################
    harm = ''.join(row[["harm"]].values)
    harm = re.sub('[()]', '', harm) # process string
    key = ''.join(row[["key"]].values)
    key_harm = key + ":" + harm
    #print(key_harm)
    try:
        chord_label = inputRN(key_harm)["Chord label"]
    except:
        continue # pass this onset slice if RN is not recognizable

    ######################################## Process window #####################################
    this_mea = ''.join(row[['measure']].values)
    if this_mea == '.':
        this_mea = 1
    else:
        this_mea = int(re.findall("\d+", this_mea)[0])
    
    if this_mea != pre_mea and this_mea % mea_window_size == 1:
        #print("coming_mea", this_mea)
        #print("new window")

        cur_window_list.append(window_onsets_list)
        cur_window_list.append(window_chords_list)
        #print("added", cur_window_list)
        all_windows.append(cur_window_list)
        
        cur_window_list = []
        window_chords_list = []
        window_onsets_list = []

    window_onsets_list.append(this_onset_vec)
    if not window_chords_list:
        window_chords_list.append(chord_label)
    elif window_chords_list and chord_label != window_chords_list[-1]:
        window_chords_list.append(chord_label)
    pre_mea = this_mea

## Append the last window
cur_window_list.append(window_onsets_list)
cur_window_list.append(window_chords_list)
all_windows.append(cur_window_list)
#print(all_windows)

SyntaxError: invalid syntax (<ipython-input-69-5de0625e0198>, line 4)

Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry.

In [9]:
print(inputRN ("g:Cad64"))

{'Current key': 'G minor', 'Tonicized key': 'None', 'Intervallic construction': 'Gm3P5', 'Inversion': '2', 'Chord label': 'G minor', 'Default function': 'tonic', 'Chord pitches': "('G', 'Bb', 'D')", 'Chord pitch classes': '(7, 10, 2)'}
